## Libraries and Paths

In [49]:
import os
import re
import sys
import numpy as np
import pandas as pd
import string
import re


import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import common_corpus, common_dictionary
import gensim


import preprocessor as p
from preprocessor.api import clean


from wordcloud import WordCloud


import matplotlib.pyplot as plt

# TODO: before saving the script try cleaning with RT @... and other punctiation stuff 
# may be # can stay it's a bit problematic


In [42]:
DATA_DIR = "../Data"
TWEETS_PATH = os.path.join(DATA_DIR, 'tweets')
TREND_PATH = os.path.join(DATA_DIR, 'trends')
SAVE_PATH = os.path.join(DATA_DIR, 'save')
os.listdir(SAVE_PATH)[:5]

['2019-07-26_trends.csv',
 '2019-07-18_trends.csv',
 '2019-07-20_trends.csv',
 '2019-07-07_trends.csv',
 '2019-07-12_trends.csv']

## Text Cleaning

In [3]:
df = pd.read_csv( os.path.join(SAVE_PATH,"2019-07-26_trends.csv"),
                header=0, usecols=[4,5,6,7], parse_dates=['trend_date'])
#Take only english ones
df = df[df.lang == "en"]
#Trend_date is not necessary now
df.drop(["lang","trend_date"], inplace=True, axis=1)

In [4]:
remove_digits = str.maketrans('', '', string.digits)
exclude = '[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
non_ascii = re.compile(r'[^\x00-\x7F]+')
#p.set_options(p.OPT.URL, p.OPT.EMOJI)

In [5]:
df['trend'] = df['trend'].map(lambda x : x.lower())
df['trend'] = df['trend'].map(lambda x : x.translate(remove_digits))
df['trend'] = df['trend'].map(lambda x : re.sub(str(exclude), '', x))    


df['text'] = df['text'].map(lambda x : x.lower())
df['text'] = df['text'].map(lambda x : clean(x))
df['text'] = df['text'].map(lambda x : x.translate(remove_digits))
df['text'] = df['text'].map(lambda x : re.sub(str(exclude), '', x))    
df['text'] = df['text'].map(lambda x : re.sub(non_ascii, '', x))

In [6]:
df = df.groupby(['trend'])['text']\
            .apply(lambda x: ','.join(x)).reset_index()
df

,trend,text
0,abel,rt abel and rihanna are doing everything but ...
1,adheera,"rt unveiling from on july th,rt hes back fro..."
2,adlive,rt adlive zero cast dates and locations annou...
3,aflbluescrows,night time is the right time to buy a for the ...
4,afleaglesnorth,rt problems for the cowboys already ezekiel e...
...,...,...
617,바이나인디어나인의우주,rt we by chart is here to help the trainees i...
618,세훈찬열부르면돼,rt sehun was the one who reached his hand out...
619,우리만믿어엑스원,"rt fighting for your debut,rt cant wait for ..."
620,유벤투스,rt camera founder found hidden camera now yo...


In [71]:
df.loc[2,:]['text']

'rt  adlive zero cast dates and locations announced for this years edition'

In [8]:
#nltk.download('punkt')
tokenized_df =  df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
tokenized_df

0      [rt, abel, and, rihanna, are, doing, everythin...
1      [rt, unveiling, from, on, july, th, ,, rt, hes...
2      [rt, adlive, zero, cast, dates, and, locations...
3      [night, time, is, the, right, time, to, buy, a...
4      [rt, problems, for, the, cowboys, already, eze...
                             ...                        
617    [rt, we, by, chart, is, here, to, help, the, t...
618    [rt, sehun, was, the, one, who, reached, his, ...
619    [rt, fighting, for, your, debut, ,, rt, cant, ...
620    [rt, camera, founder, found, hidden, camera, n...
621    [boss, ,, rt, camera, founder, found, hidden, ...
Length: 622, dtype: object

In [9]:
def semmatize_text(text):
    return [ps.stem(w)  for w in text if len(w)>3]
ps = PorterStemmer() 
stemmed_dataset = tokenized_df.apply(semmatize_text)
stemmed_dataset

0      [abel, rihanna, do, everyth, drop, album, abel...
1      [unveil, from, juli, back, from, dead, unveil,...
2      [adliv, zero, cast, date, locat, announc, thi,...
3      [night, time, right, time, celebr, here, somet...
4      [problem, cowboy, alreadi, ezekiel, elliott, r...
                             ...                        
617    [chart, here, help, traine, achiev, their, dre...
618    [sehun, reach, hand, love, love, sehun, reach,...
619    [fight, your, debut, cant, wait, your, redebut...
620    [camera, founder, found, hidden, camera, easil...
621    [boss, camera, founder, found, hidden, camera,...
Length: 622, dtype: object

## Exploratory Analysis

In [13]:
dataset_words=''.join(list(str(stemmed_dataset.values)))
wordcloud = WordCloud(width = 800, height = 500, 
                background_color ='white',  
                min_font_size = 10).generate(dataset_words) 

# plt.figure(figsize = (5, 5), facecolor = None) 
# plt.imshow(wordcloud) 
# plt.axis("off") 
# plt.tight_layout(pad = 0) 
  
# plt.show()

In [15]:
dictionary_of_words = gensim.corpora.Dictionary(stemmed_dataset)
print(len(dictionary_of_words))

25155


In [35]:
word_corpus = [dictionary_of_words.doc2bow(word) for word in stemmed_dataset]

for corp in word_corpus[:1]:
    for id, freq in corp[:5]:
        print(dictionary_of_words[id],freq)

dictionary_of_words[15]

abel 34
advic 1
aint 1
album 25
analysi 1


'call'

## LDA Model Initial Trial

In [36]:
lda_model = models.ldamodel.LdaModel(corpus=word_corpus,
                                           id2word=dictionary_of_words,
                                           num_topics=30, 
                                           random_state=101,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [38]:
# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} \nWords: {}'.format(idx, topic))

In [43]:
coherence_val = CoherenceModel(model=lda_model, texts=stemmed_dataset, 
                               dictionary=dictionary_of_words, coherence='c_v').get_coherence()
print('Coherence Score: ', coherence_val)



Coherence Score:  0.5331569891775539


#### Best Score is at num_topics=20

In [44]:
lda_models=[]
coherence_values = []
for topic_number in range(5,50,5):
    lda_model = models.ldamodel.LdaModel(corpus=word_corpus,
                                           id2word=dictionary_of_words,
                                           num_topics=topic_number, 
                                           random_state=101,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)
    lda_models.append(lda_model)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=stemmed_dataset, 
                                         dictionary=dictionary_of_words, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)
    print("number of topics ",topic_number,"coherence_value :" , coherence_lda)

number of topics  5 coherence_value : 0.4322777314653199
number of topics  10 coherence_value : 0.5775055519448571
number of topics  15 coherence_value : 0.5608834659197979
number of topics  20 coherence_value : 0.5734361394539241
number of topics  25 coherence_value : 0.5684044645318396
number of topics  30 coherence_value : 0.5331569891775539
number of topics  35 coherence_value : 0.5607330212208015
number of topics  40 coherence_value : 0.5323336344312884
number of topics  45 coherence_value : 0.5174372823206751


In [46]:
# Model with the best coherence_value
lda_model_20 = models.ldamodel.LdaModel(corpus=word_corpus,
                                           id2word=dictionary_of_words,
                                           num_topics=20, 
                                           random_state=1,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [48]:
# Compute Coherence Score
cohr_val = CoherenceModel(model=lda_model_20, texts=stemmed_dataset, dictionary=dictionary_of_words, coherence='c_v').get_coherence()

print('\nCoherence Score: ', cohr_val)


Coherence Score:  0.5945144461036688


## LDAMulticore Model

In [52]:
lda_multicore_model = models.ldamulticore.LdaMulticore(corpus=word_corpus, 
                                                              num_topics=20, 
                                                              id2word=dictionary_of_words,                                                             
                                                              chunksize=100, 
                                                              passes=50,                                
                                                              alpha='symmetric',
                                                              eta=0.1,
                                                              decay=0.5, 
                                                              offset=1.0, 
                                                              gamma_threshold=0.001,
                                                              random_state=101,
                                                              minimum_probability=0.01,
                                                              minimum_phi_value=0.01,
                                                              per_word_topics=False)



In [53]:
# Compute Coherence Score
cohr_lda_multicore_model1 = CoherenceModel(model=lda_multicore_model, texts=stemmed_dataset, 
                                           dictionary=dictionary_of_words, coherence='c_v').get_coherence()
print('\nCoherence Score: ', cohr_lda_multicore_model1)


Coherence Score:  0.48426339501658616


## Evaluation

In [80]:
v = lda_model_20[word_corpus[2]]
print(type(lda_model[word_corpus[2]]))
z=sorted(v[0], key=lambda tup: -1*tup[1])
print(z)

<class 'tuple'>
[(15, 0.44074607), (4, 0.3553429), (11, 0.12210046), (3, 0.011222988), (0, 0.010196379)]


In [119]:
lda_model_20[word_corpus[500]][0]

[(4, 0.7901357),
 (6, 0.10349235),
 (11, 0.021349145),
 (13, 0.05406458),
 (14, 0.019050092)]

In [82]:
for  index,score in sorted(lda_model_20[word_corpus[2]][0], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4408090114593506	 
Topic: 0.040*"mitch" + 0.039*"elect" + 0.031*"mcconnel" + 0.029*"russian" + 0.025*"vote" + 0.020*"traitor" + 0.020*"refus" + 0.018*"secur" + 0.018*"russia" + 0.017*"democraci"

Score: 0.3552166521549225	 
Topic: 0.023*"thi" + 0.020*"with" + 0.012*"have" + 0.011*"your" + 0.010*"more" + 0.009*"that" + 0.009*"will" + 0.008*"from" + 0.008*"time" + 0.007*"here"

Score: 0.1221638098359108	 
Topic: 0.054*"regim" + 0.028*"rihanna" + 0.025*"abel" + 0.024*"everyth" + 0.022*"ann" + 0.021*"do" + 0.021*"drop" + 0.021*"mike" + 0.014*"hathaway" + 0.013*"essenc"

Score: 0.011222981847822666	 
Topic: 0.037*"morn" + 0.033*"sunday" + 0.020*"atlant" + 0.009*"extra" + 0.005*"endlich" + 0.005*"allein" + 0.004*"queremo" + 0.004*"ampkendricklamar" + 0.003*"armin" + 0.003*"kygo"

Score: 0.010196379385888577	 
Topic: 0.049*"bernal" + 0.036*"egan" + 0.034*"stage" + 0.034*"first" + 0.034*"yellow" + 0.032*"jersey" + 0.030*"pinot" + 0.025*"best" + 0.023*"tour" + 0.022*"franc"


In [89]:
topics = lda_model_20.show_topics()
topics[]

10

In [112]:
lda_model_20.print_topics(1) # list of tuples where tuple[0] = topicID tuple[1] = topicWords
# what is written inside is index 
# in the list they are ordered but not 1,2,3 I guess based on size

[(4,
  '0.023*"thi" + 0.017*"that" + 0.015*"with" + 0.015*"your" + 0.012*"have" + 0.009*"will" + 0.008*"from" + 0.008*"like" + 0.007*"love" + 0.007*"about"')]

In [115]:
for idx, topic in lda_model_20.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.097*"face" + 0.023*"famou" + 0.018*"that" + 0.013*"your" + 0.013*"when" + 0.012*"like" + 0.012*"look" + 0.010*"they" + 0.009*"aid" + 0.009*"trump"
Topic: 1 
Words: 0.000*"hughbrownston" + 0.000*"hyphynhumbl" + 0.000*"hillaryclitori" + 0.000*"honeybunnyb" + 0.000*"honmychest" + 0.000*"hellosh" + 0.000*"itslexxiebabi" + 0.000*"isaiahheath" + 0.000*"heddicundl" + 0.000*"illswagg"
Topic: 2 
Words: 0.126*"lucki" + 0.067*"brown" + 0.065*"chri" + 0.042*"video" + 0.041*"guidanc" + 0.032*"drake" + 0.018*"again" + 0.017*"see" + 0.017*"front" + 0.016*"march"
Topic: 3 
Words: 0.046*"what" + 0.046*"fuck" + 0.042*"scene" + 0.027*"that" + 0.026*"toni" + 0.026*"cover" + 0.026*"villag" + 0.025*"thi" + 0.023*"show" + 0.022*"they"
Topic: 4 
Words: 0.023*"thi" + 0.017*"that" + 0.015*"with" + 0.015*"your" + 0.012*"have" + 0.009*"will" + 0.008*"from" + 0.008*"like" + 0.007*"love" + 0.007*"about"
Topic: 5 
Words: 0.069*"madrid" + 0.061*"real" + 0.019*"goal" + 0.018*"costa" + 0.018*"atletic

## Test for Trend-Key_Words

In [120]:
# Model for key-words finding
lda_model_key_words = models.ldamodel.LdaModel(corpus=word_corpus,
                                           id2word=dictionary_of_words,
                                           num_topics=600, 
                                           random_state=1,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [121]:
# Compute Coherence Score
cohr_lda_multicore_model1 = CoherenceModel(model=lda_multicore_model, texts=stemmed_dataset, 
                                           dictionary=dictionary_of_words, coherence='c_v').get_coherence()
print('\nCoherence Score: ', cohr_lda_multicore_model1)


Coherence Score:  0.48426339501658616


In [122]:
lda_model_key_words.print_topics(5)

[(242,
  '0.000*"hughbrownston" + 0.000*"hyphynhumbl" + 0.000*"hillaryclitori" + 0.000*"honeybunnyb" + 0.000*"honmychest" + 0.000*"hellosh" + 0.000*"itslexxiebabi" + 0.000*"isaiahheath" + 0.000*"heddicundl" + 0.000*"illswagg"'),
 (54,
  '0.000*"hughbrownston" + 0.000*"hyphynhumbl" + 0.000*"hillaryclitori" + 0.000*"honeybunnyb" + 0.000*"honmychest" + 0.000*"hellosh" + 0.000*"itslexxiebabi" + 0.000*"isaiahheath" + 0.000*"heddicundl" + 0.000*"illswagg"'),
 (291,
  '0.059*"mitch" + 0.050*"mcconnel" + 0.034*"seoul" + 0.030*"traitor" + 0.029*"refus" + 0.015*"biggest" + 0.015*"remix" + 0.015*"singapor" + 0.013*"that" + 0.013*"moscow"'),
 (415,
  '0.029*"thi" + 0.019*"will" + 0.015*"pleas" + 0.014*"let" + 0.013*"know" + 0.013*"them" + 0.013*"love" + 0.013*"that" + 0.012*"with" + 0.012*"well"'),
 (151,
  '0.024*"that" + 0.020*"thi" + 0.020*"have" + 0.017*"with" + 0.016*"your" + 0.010*"from" + 0.009*"will" + 0.008*"dont" + 0.008*"want" + 0.008*"make"')]